## Assigment 2 Data Mining

Your task is to predict what hotel properties that result from a search of a user, the user is
most likely to click on.


The core task is "to predict what hotel a user is most likely to book"

**OUTCOME VARIABLE** is a combination of:

- `click_bool`: Whether the user clicked on that specific property (1=yes, 0=no).
- `booking_bool`: Whether the user booked that specific property (1=yes, 0=no).


**Evaluation Metric** (NDCG@5)
- Relevance 5: User booked the hotel (`booking_bool = 1`).
- Relevance 1: User clicked but did not book (`click_bool = 1` AND `booking_bool = 0`).
- Relevance 0: User neither clicked nor booked (`click_bool = 0` AND `booking_bool = 0`).

VIEW TOP RESULTS:
https://www.kaggle.com/competitions/expedia-personalized-sort/overview

FOLLOW-ALLONG KAGGLE TUTORIAL FOR A DESCENT SUBMISSION:
https://www.dataquest.io/blog/kaggle-tutorial/
https://www.studeersnel.nl/nl/document/vrije-universiteit-amsterdam/data-mining-techniques/expedia-assignment/95305972 old submission vu

**TODO**: 
- DESCRIBE TOP 5 SOLUTIONS
- Describe more recent submissions outside the competition

1. uses LambdaMART: a type of boosting algo
2. ensemble of 26 Gradient BoostingMachines (GBM)
3. LambdaMART
4. LambdaMART
5. 


In [2]:
import pandas as pd

In [ ]:
hotels = pd.read_csv('data/test_set_VU_DM.csv')

hotels

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,comp5_rate_percent_diff,comp6_rate,comp6_inv,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff
0,1,2013-02-02 15:27:40,24,216,NaN,NaN,219,3180,3,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2013-02-02 15:27:40,24,216,NaN,NaN,219,5543,3,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2013-02-02 15:27:40,24,216,NaN,NaN,219,14142,2,3.5,...,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2013-02-02 15:27:40,24,216,NaN,NaN,219,22393,3,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,2013-02-02 15:27:40,24,216,NaN,NaN,219,24194,3,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4959178,332787,2013-05-21 11:06:37,24,216,NaN,NaN,117,32019,4,3.5,...,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4959179,332787,2013-05-21 11:06:37,24,216,NaN,NaN,117,33959,4,3.0,...,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4959180,332787,2013-05-21 11:06:37,24,216,NaN,NaN,117,35240,4,0.0,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4959181,332787,2013-05-21 11:06:37,24,216,NaN,NaN,117,94437,4,0.0,...,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
#hotels.dtypes
hotels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4959183 entries, 0 to 4959182
Data columns (total 50 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   srch_id                      int64  
 1   date_time                    object 
 2   site_id                      int64  
 3   visitor_location_country_id  int64  
 4   visitor_hist_starrating      float64
 5   visitor_hist_adr_usd         float64
 6   prop_country_id              int64  
 7   prop_id                      int64  
 8   prop_starrating              int64  
 9   prop_review_score            float64
 10  prop_brand_bool              int64  
 11  prop_location_score1         float64
 12  prop_location_score2         float64
 13  prop_log_historical_price    float64
 14  price_usd                    float64
 15  promotion_flag               int64  
 16  srch_destination_id          int64  
 17  srch_length_of_stay          int64  
 18  srch_booking_window          int64  
 19  

Each line in the dataset represents a combination of a search query by a user with one specific
hotel property that was shown as part of the results. 

Of course, a list of hotels is presented to the user (and hence, there are multiple rules describing a single search).

In [5]:
hotels.iloc[0,:]

srch_id                                          1
date_time                      2013-02-02 15:27:40
site_id                                         24
visitor_location_country_id                    216
visitor_hist_starrating                        NaN
visitor_hist_adr_usd                           NaN
prop_country_id                                219
prop_id                                       3180
prop_starrating                                  3
prop_review_score                              4.5
prop_brand_bool                                  1
prop_location_score1                          2.94
prop_location_score2                        0.0691
prop_log_historical_price                     5.03
price_usd                                    119.0
promotion_flag                                   0
srch_destination_id                          19222
srch_length_of_stay                              1
srch_booking_window                             10
srch_adults_count              

In [6]:
hotels.describe()


,srch_id,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,...,comp5_rate_percent_diff,comp6_rate,comp6_inv,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff
count,4.959183e+06,4.959183e+06,4.959183e+06,253431.000000,254624.000000,4.959183e+06,4.959183e+06,4.959183e+06,4.951917e+06,4.959183e+06,...,839907.000000,242330.000000,263169.000000,9.713800e+04,315729.000000,356753.000000,139323.000000,1.902389e+06,1.972885e+06,610977.000000
mean,1.666460e+05,9.979214e+00,1.752504e+02,3.374933,177.150730,1.738371e+02,7.008111e+04,3.182542e+00,3.777628e+00,6.339091e-01,...,26.910952,0.129617,0.075347,2.872014e+01,0.148152,0.081317,20.846400,-6.054913e-02,9.815575e-03,23.994402
std,9.614992e+04,7.667722e+00,6.590284e+01,0.691920,109.592336,6.835006e+01,4.061363e+04,1.049358e+00,1.051283e+00,4.817348e-01,...,1019.245372,0.560591,0.301452,3.611865e+03,0.579293,0.315606,184.562247,4.693195e-01,2.034170e-01,994.201341
min,1.000000e+00,1.000000e+00,1.000000e+00,1.000000,0.000000,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,2.000000,-1.000000,-1.000000,2.000000e+00,-1.000000,-1.000000,2.000000,-1.000000e+00,-1.000000e+00,2.000000
25%,8.331150e+04,5.000000e+00,1.000000e+02,2.930000,111.010000,1.000000e+02,3.502100e+04,3.000000e+00,3.500000e+00,0.000000e+00,...,7.000000,0.000000,0.000000,6.000000e+00,0.000000,0.000000,7.000000,0.000000e+00,0.000000e+00,7.000000
50%,1.670950e+05,5.000000e+00,2.190000e+02,3.430000,151.390000,2.190000e+02,6.960700e+04,3.000000e+00,4.000000e+00,1.000000e+00,...,12.000000,0.000000,0.000000,1.100000e+01,0.000000,0.000000,12.000000,0.000000e+00,0.000000e+00,11.000000
75%,2.499660e+05,1.400000e+01,2.190000e+02,3.950000,215.380000,2.190000e+02,1.051790e+05,4.000000e+00,4.500000e+00,1.000000e+00,...,21.000000,0.000000,0.000000,1.900000e+01,1.000000,0.000000,20.000000,0.000000e+00,0.000000e+00,17.000000
max,3.327870e+05,3.400000e+01,2.310000e+02,5.000000,2768.930000,2.300000e+02,1.408210e+05,5.000000e+00,5.000000e+00,1.000000e+00,...,269379.000000,1.000000,1.000000,1.125686e+06,1.000000,1.000000,30216.000000,1.000000e+00,1.000000e+00,144900.000000


In [10]:
hotels.isnull().sum()

srch_id                              0
date_time                            0
site_id                              0
visitor_location_country_id          0
visitor_hist_starrating        4705752
visitor_hist_adr_usd           4704559
prop_country_id                      0
prop_id                              0
prop_starrating                      0
prop_review_score                 7266
prop_brand_bool                      0
prop_location_score1                 0
prop_location_score2           1088032
prop_log_historical_price            0
price_usd                            0
promotion_flag                       0
srch_destination_id                  0
srch_length_of_stay                  0
srch_booking_window                  0
srch_adults_count                    0
srch_children_count                  0
srch_room_count                      0
srch_saturday_night_bool             0
srch_query_affinity_score      4641025
orig_destination_distance      1608679
random_bool              

In [ ]:
data = {
    'item':['apple','banana','cucumber','eggplant','citrus','lamp']
}

d_map = {['apple','banana','citrus'],
         'vegetable':['cucumber','eggplant'],
         'non-food':['lamp']}

df = pd.DataFrame(data)

df['type'] = df['item'].map(d_map)

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
5    NaN
Name: item, dtype: object